Puzzle
----

https://thefiddler.substack.com/p/how-greedily-can-you-mow-the-lawn

Fiddler "Solution"
----

I think it is 4 passes.

https://www.desmos.com/calculator/iceqgnygwq

Extra Credit Solution
---

Desmos approach doesn't really work, though based just on the figure ( https://www.desmos.com/3d/xytaslvftu ), I am guessing 13 passes:
- 3 along the axes: X, Y, Z
- 6 at 45 degress from each pair of axes: 2 each in XY, XZ, YZ.
- 4 at 45 degress to all 3 axes.

I am going to try to code up something also, but hopefully without too much effort.

In [7]:
# This is a 3D problem, so we have 3 dimensions
# Number of grid points in each dimension
N = 200

# T is the number of cylinders to test in each pass.
T = 600

# Generate a grid of points in 3D space
# Each point is one entry in a 3D array.
# Initialize an empty list to hold the grid points
# We will use a nested loop to create the grid points while initialzing the sphere.
grid = []
#print("Grid points:", grid)

from functools import cache

# Distance function to calculate the distance between two points in 3D space
# This function takes the coordinates of two points and returns the distance between them
def distance_pt_pt(x1, y1, z1, x2, y2, z2):
    return ((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)**0.5

@cache
# Calculate the parameters of the line defined by two points in 3D space
def line_params_from_points(x1, y1, z1, x2, y2, z2):
    # Calculate the direction vector of the line
    dx = x2 - x1
    dy = y2 - y1
    dz = z2 - z1
    # Calculate the length of the line segment
    line_length_sq = dx**2 + dy**2 + dz**2
    return dx, dy, dz, line_length_sq

# Calculate the distance between a point and a line in 3D space
# The line is defined by 2 points (x1, y1, z1) and (x2, y2, z2)
def distance_pt_line(x, y, z, x1, y1, z1, x2, y2, z2):
    dx, dy, dz, line_length_sq = line_params_from_points(x1, y1, z1, x2, y2, z2)
    # If the line length is zero, return the distance to one of the points
    if line_length_sq == 0:
        return distance_pt_pt(x, y, z, x1, y1, z1)
    # Calculate the projection of the point onto the line
    t = ((x-x1)*dx + (y-y1)*dy + (z-z1)*dz) / line_length_sq
    # Clamp t to the range [0, 1]
    #t = max(0, min(1, t))
    # Calculate the closest point on the line to the point
    closest_x = x1 + t*dx
    closest_y = y1 + t*dy
    closest_z = z1 + t*dz
    # Return the distance from the point to the closest point on the line
    return distance_pt_pt(x, y, z, closest_x, closest_y, closest_z)

# Not totally certain about the distance_pt_line function, but it seems to be working.
# Test the distance function with some known points that are not on the line.
assert (distance_pt_line(0, 0, 0, 1, 1, 1, 2, 2, 2) == 0)
assert (distance_pt_line(0, 0, 1, 5, 0, 0, 7, 0, 0) == 1)
assert (distance_pt_line(0, 2, 0, 5, 0, 0, 7, 0, 0) == 2)

# Create a version of the distance function that uses the line parameters
# This is a more efficient version that avoids recalculating the line parameters
def distance_pt_line_alt(x, y, z, x1, y1, z1, dx, dy, dz, line_length_sq):
    # If the line length is zero, return the distance to one of the points
    if line_length_sq == 0:
        return distance_pt_pt(x, y, z, x1, y1, z1)
    # Calculate the projection of the point onto the line
    t = ((x-x1)*dx + (y-y1)*dy + (z-z1)*dz) / line_length_sq
    # Clamp t to the range [0, 1]
    #t = max(0, min(1, t))
    # Calculate the closest point on the line to the point
    closest_x = x1 + t*dx
    closest_y = y1 + t*dy
    closest_z = z1 + t*dz
    # Return the distance from the point to the closest point on the line
    return distance_pt_pt(x, y, z, closest_x, closest_y, closest_z)

# Insert sphere into the grid
def insert_sphere(grid, center_x=0, center_y=0, center_z=0, radius=1, two_dimensional=False):
    k_start = 1
    k_limit = N + 1
    if two_dimensional:
        k_start = int(N/2)
        k_limit = k_start + 1
    # Loop through each point in the grid
    for i in range(N+1):
        x = (2*i-N)/N
        for j in range(N+1):
            y = (2*j-N)/N
            for k in range(k_start,k_limit):
                z = (2*k-N)/N
                # Calculate the distance from the center of the sphere
                distance = distance_pt_pt(x, y, z, center_x, center_y, center_z)
                # If the distance is less than the radius, set the value to 1
                if distance <= radius:
                    grid.append((x, y, z))
    # Return the grid with the sphere points
    return grid

# Test the intersection of a cylinder  with the grid.
# The cylinder is given by 2 points and radius = 0.5
def test_cylinder(grid, x1,y1,z1,x2,y2,z2, radius=0.5, count=True, clear=False):
    cnt = 0
    new_grid = []
    dx, dy, dz, line_length_sq = line_params_from_points(x1, y1, z1, x2, y2, z2)

    # Loop through each point in the grid
    for x,y,z in grid:
        # Calculate the distance from the point to the line
        distance = distance_pt_line_alt(x, y, z, x1, y1, z1, dx, dy, dz, line_length_sq)
        # If the distance is less than the radius, count the point or clear it
        if (clear):
            if distance > radius:
                new_grid.append((x, y, z))
        elif count:
            if distance <= radius:
                cnt += 1
        
    if count:
        return cnt
    elif clear:        
        return new_grid
    else:
        return

import random
def pick_pair(grid):
    # Pick a random pair of points in the grid, distinct to the extent possible.
    i1 = random.randint(0, len(grid)-1)
    i2 = random.randint(0, len(grid)-1)
    if i1 == i2:
        i2 = i1 - 1
    return grid[i1], grid[i2]

In [8]:
grid = []
insert_sphere(grid, 0, 0, 0, 1, two_dimensional=False)
print("Grid points:", len(grid))

pass_num = 1
while len(grid) > 0:
    print("Pass", pass_num, "Remaining points:", len(grid))
    best_count = 0
    best_p1, best_p2 = None, None
    if pass_num == 1:
        p1 = (-1, 0, 0)
        p2 = (1, 0, 0)
        count = test_cylinder(grid, p1[0], p1[1], p1[2], p2[0], p2[1], p2[2], count=True, clear=False)
        best_count = count
        best_p1, best_p2 = p1, p2
    else:
        for i in range(T):
            # Pick a random pair of points in the grid
            p1, p2 = pick_pair(grid)
            # This is good that the cylinder definitely intersects the sphere, 
            # but it is not good in that the optimal cylinder may lie in beteween points.
            
            # Test the cylinder between the two points
            count = test_cylinder(grid, p1[0], p1[1], p1[2], p2[0], p2[1], p2[2], count=True, clear=False)
            if count > best_count:
                best_count = count
                best_p1, best_p2 = p1, p2
    print("Pass:", pass_num, "Best pair:", best_p1, best_p2, "with count", best_count)
    grid = test_cylinder(grid, best_p1[0], best_p1[1], best_p1[2], best_p2[0], best_p2[1], best_p2[2], count=False, clear=True)
    pass_num += 1

print("Final pass:", pass_num - 1, "Remaining points:", len(grid))
print("Final grid:", grid)

Grid points: 4187856
Pass 1 Remaining points: 4187856
Pass: 1 Best pair: (-1, 0, 0) (1, 0, 0) with count 1466373
Pass 2 Remaining points: 2721483
Pass: 2 Best pair: (0.01, -0.27, -0.46) (-0.02, 0.35, -0.51) with count 920838
Pass 3 Remaining points: 1800645
Pass: 3 Best pair: (0.03, -0.78, 0.46) (-0.08, 0.39, 0.42) with count 898290
Pass 4 Remaining points: 902355
Pass: 4 Best pair: (0.42, 0.38, 0.72) (0.46, 0.17, -0.69) with count 304812
Pass 5 Remaining points: 597543
Pass: 5 Best pair: (-0.39, -0.33, -0.81) (-0.58, -0.32, 0.67) with count 249045
Pass 6 Remaining points: 348498
Pass: 6 Best pair: (0.59, -0.46, 0.26) (-0.71, 0.5, -0.09) with count 199195
Pass 7 Remaining points: 149303
Pass: 7 Best pair: (-0.55, 0.29, 0.72) (0.65, -0.57, -0.35) with count 69771
Pass 8 Remaining points: 79532
Pass: 8 Best pair: (0.32, -0.11, 0.88) (0.52, 0.79, 0.0) with count 49193
Pass 9 Remaining points: 30339
Pass: 9 Best pair: (-0.55, 0.19, -0.54) (-0.61, 0.3, -0.61) with count 28296
Pass 10 Remain

N=100, T=200 - 13 passes. Oooh, that's tantalizingly close to the handwavy answer. It takes a few minutes to run because we initially have 500k points.

N=125, T=250 - 12 passes. 15 minutes to run because we initially have 1M points.

150/300 - 14 passes. 20 minutes.

150/300 - 11 passes. 19 minutes.

151/301 - 12 passes. 21 minutes.

Optimized and corrected some code. Fixed the first pass to be along the x axis, speeding things up.

151/301 - 12 passes , 12 minutes

161/400 - 13 passes. 18 minutes.

161/400 - 12 passes, 26 minutes.

161/400 - 13 passes.

Digression to note that 2D case does yield 4, a few times over.

12 vs 13 is not clear.
Going to do a big run overnight to settle the matter! Hopefully ... :)

Actually, the much touted 301/1000 overnight run failed because of a bug (Excessively large printout) and because the computer went to sleep. :( :)

13 is probably the best guess at the moment.

N=160, T=500 - 13 again. 28 minutes.

200/600 - 11 passes. 56 minutes.

200/600 - 13 again. 66 minutes.

Stop here before there's any conflicting runs. :)